# 1. Creation the SQNce-db file with all tables 
<a class="anchor" id="section1"></a>

In [1]:
import os
import gzip
import sqlite3
import zlib
from sqlite3 import Error
import pandas as pd
import numpy as np
from collections import OrderedDict

from Bio import SeqIO 
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

In [2]:
# Current implementation requires re-parsing of all the input files to create SQNce
# TODO SQNce update functions to parse input data only if not previously included 

# Establish connection with SQNce.db, generating a new SQLite3 database if needed
def sql_connection():
    try:
        con = sqlite3.connect('SQNce.db')
        print("Connection established.")
        return(con)
    except Error:
        print(Error)

# if os.path.exists("SQNce.db"): os.remove("SQNce.db")
con = sql_connection()

Connection established.


### Table of Contents
<a class="anchor" id="section2"></a>
* [1. Creation the SQNce-db file with all tables](#section1)
* [2. Populate SQNce-db with the required data](#section2)
* [Insert GO Terms](#GO)
* [Insert gene genomic coordinates](#coordinates)
* [Insert protein sequences](#proteins)
* [Insert gene family annotations](#families)
* [Insert best blast hists (BBHs)](#BBHs)
* [Insert OrthoFinder orthogroups](#orthogroups)
* [Insert promoter sequences](#promoters)
* [Gene annotation insert](#annotations)
* [Insert gene symbols](#symbols)
* [Insert RNA-seq Files](#RNAseq)

# Description
* SQNce database is initiated using predifined TSV files
* TSV either contain the input data or reference input files to parse


# To-do
* Documentation and nomenclature
* Standarize the column ids across the different tables
* Add a 'force' option to overwrite existing table with new parsed data
* Re-index columns if a table is updated

# 2. Populate SQNce-db with the required data

<a class="anchor" id="GO"></a>
# Insert GO Terms
* [Go back to section 2](#section2)

In [ ]:
# Add the GO database OBO file to keep the GO annotations
df = pd.read_csv("inputs/GO/go-basic.csv").drop("Unnamed: 0", axis=1)
df.columns = ["GO_id", "GO_short", "process", "GO_long"]
df.to_sql('GO_basic', con, if_exists='replace', index=False)

con.cursor().execute("CREATE INDEX GO_basic_id ON GO_basic(GO_id)")
con.commit()

In [ ]:
con.cursor().execute("""CREATE TABLE IF NOT EXISTS genomes(
                     genome_id text,
                     species_name text,
                     genotype_id text,
                     source_id text,
                     file_name text,
                     gene_number integer)""")
con.commit()

df = pd.read_csv("init/genomes.tsv", sep="\t")
for row in df.values:
    
    # Don't update update table if genome id already exists
    if con.cursor().execute('''SELECT COUNT(*) FROM genomes WHERE genome_id =  ?''', (row[0],)).fetchall()[0][0]!=0:
        continue
    
    # If from phytozome use the annotation table to read number of genes
        annot = pd.read_csv("inputs/annotations/"+row[4], sep="\t")
        annot = annot.drop_duplicates(subset="locusName")

        con.cursor().execute("""INSERT INTO genomes(
                             genome_id, species_name, genotype_id,
                             source_id, file_name, gene_number) 
                             VALUES(?, ?, ?, ?, ?, ?)""", row+[annot.shape[0]])
        con.commit()

In [ ]:
con.cursor().execute("""CREATE TABLE IF NOT EXISTS gene_GOs(
                     gene_id text, 
                     GO_id text, 
                     GO_count integer,
                     genome_id text,
                     source_id text)""")

df = pd.read_csv("init/GO_files.tsv", sep="\t")
for row in df.values:
    # Check annotation column
    long_list = [] # have a row for each gene-GO combination
    if row[3] == "Phytozome": # currently only support phytozome GO annotations
        
        # Don't update update table if genome id already exists
        if con.cursor().execute('''SELECT COUNT(*) FROM gene_GOs WHERE genome_id =  ?  ''', (row[0],)).fetchall()[0][0]!=0:
            continue

        annot = pd.read_csv("inputs/annotations/"+row[4], sep="\t")[['locusName', 'GO']]
        # I am assuming that all variants have the same GO term but should double check
        annot = annot.drop_duplicates(subset="locusName")
        

        # https://stackoverflow.com/questions/41244981/how-to-extract-comma-separated-values-to-individual-rows-in-pandas
        annot = annot.set_index('locusName').GO.str.split(',', expand=True).stack().reset_index('locusName')
        annot.columns = ["gene_id", "GO"]
        annot = annot.merge(annot.groupby("GO").count().reset_index(), on="GO", how="left")
        annot["genome_id"] = row[0]
        annot["source_id"] = row[3]

        con.cursor().executemany("""INSERT INTO gene_GOs(
                                 gene_id, GO_id, GO_count, genome_id, source_id)  
                                 VALUES(?,?,?,?,?)""", annot.values.tolist())
con.commit()

# Create a secondary key on the name column
cursorObj = con.cursor()
cursorObj.execute("CREATE INDEX IF NOT EXISTS gene_GOs_index_gene_id ON gene_GOs(gene_id)")
con.commit()

<a class="anchor" id="coordinates"></a>
# Insert gene genomic coordinates
* [Go back to section 2](#section2)

In [ ]:
con.cursor().execute("""CREATE TABLE IF NOT EXISTS gene_coordinates(
                     gene_id text,
                     genome_id text,
                     gene_chr text,
                     gene_start integer,
                     gene_end integer,
                     gene_orientation text)""")

df = pd.read_csv("init/coordinates.tsv", sep="\t")
for row in df.values:
    # Don't update update table if genome id already exists
    if con.cursor().execute('''SELECT COUNT(*) FROM gene_coordinates WHERE genome_id =  ?  ''', (row[1],)).fetchall()[0][0]!=0:
        continue
    
    gff3_file = gzip.open("inputs/gff3/"+row[3], mode='rt')
    gene_coordinate_list = []
    for gene in gff3_file:
        gene = gene.split("\t")
        if len(gene) == 1:
            # skip the first row
            continue
        if gene[2] == "gene":
            gene_id = gene[-1].split(";")
            gene_id = [i for i in gene_id if i.startswith('Name=')][0].replace('Name=', '').replace('\n', '') # delete \n if exists
            # Append list of: gene ID, genome_id, chromsome, start, end, orientation
            gene_coordinate_list.append([gene_id, row[1], gene[0], gene[3], gene[4], gene[6]])
        else:
            continue
    con.cursor().executemany("""INSERT INTO gene_coordinates(
                         gene_id, genome_id, gene_chr, 
                         gene_start, gene_end, gene_orientation) 
                         VALUES(?, ?, ?, ?, ?, ?)""", gene_coordinate_list)

# Create a secondary key on the name column
cursorObj = con.cursor()
cursorObj.execute("CREATE INDEX IF NOT EXISTS coordinate_index_start ON gene_coordinates(genome_id, gene_chr, gene_start)")
cursorObj.execute("CREATE INDEX IF NOT EXISTS coordinate_index_end ON gene_coordinates(genome_id, gene_chr, gene_end)")
con.commit()

<a class="anchor" id="proteins"></a>
# Insert protein sequences
* [Go back to section 2](#section2)

In [3]:
# https://stackoverflow.com/questions/18219779/bulk-insert-huge-data-into-sqlite-using-python
con.cursor().execute("""CREATE TABLE IF NOT EXISTS protein_seqs(
                     protein_id text,
                     species_id text,
                     genome_id text,
                     protein_length text,
                     protein_sequence blob)""")

df = pd.read_csv("init/proteins.tsv", sep="\t")
for row in df.values.tolist():
    
    if con.cursor().execute('''SELECT COUNT(*) FROM protein_seqs WHERE genome_id =  ?  ''', (row[0],)).fetchall()[0][0]!=0:
        continue

    
    fasta_file = gzip.open("inputs/proteins/"+row[2], mode='rt')
    protein_seq_list = []
    for seq in SeqIO.parse(fasta_file, "fasta"):
        # Protein sequences are saved as as a Binary data type for compression
        protein_seq_list.append([seq.id, row[0], row[1], len(seq.seq), 
                                 sqlite3.Binary(zlib.compress(str(seq.seq).encode('utf-8')))])
    con.cursor().executemany("""INSERT INTO protein_seqs(
                     protein_id, species_id, genome_id, 
                     protein_length, protein_sequence) 
                     VALUES(?, ?, ?, ?, ?)""", protein_seq_list)

# Create keys for protein IDs with isoform and without
con.cursor().execute("CREATE INDEX IF NOT EXISTS proteins_ids_index ON protein_seqs(protein_id)")
con.commit()

<a class="anchor" id="families"></a>
# Insert gene family annotations
* [Go back to section 2](#section2)

In [ ]:
con.cursor().execute("""CREATE TABLE IF NOT EXISTS gene_families(
                     protein_id text,
                     genome_id text,
                     source_id text,
                     family_id text,
                     family_name text)""")

df = pd.read_csv("init/families.tsv", sep="\t")
for row in df.values:
    if con.cursor().execute('''SELECT COUNT(*) FROM gene_families WHERE genome_id =  ?  ''', (row[1],)).fetchall()[0][0]!=0:
        continue

    annot = pd.read_csv("inputs/families/"+row[3], sep="\t")
    annot.insert(1, 'genome', row[1])
    annot.insert(2, 'source', row[2])
    con.cursor().executemany("""INSERT INTO gene_families(
                         protein_id, genome_id,
                         source_id, family_id, family_name) 
                         VALUES(?, ?, ?, ?, ?)""", annot.values.tolist())
# Create keys for protein IDs with isoform and without
con.cursor().execute("CREATE INDEX IF NOT EXISTS family_gene_index ON gene_families(protein_id)")
con.cursor().execute("CREATE INDEX IF NOT EXISTS family_name_index ON gene_families(family_name)")
con.cursor().execute("CREATE INDEX IF NOT EXISTS family_genotype_name_index ON gene_families(genome_id, family_name)")
con.commit()   

<a class="anchor" id="BBHs"></a>
# Insert best blast hists (BBHs)
* [Go back to section 2](#section2)

In [ ]:
con.cursor().execute("""CREATE TABLE IF NOT EXISTS BBHs(
                     subject_id text,
                     query_id text,
                     bit_score integer,
                     subject_genome text,
                     query_genome text)""")

df = pd.read_csv("init/BBHs_files.tsv", sep="\t")
for row in df.values:
    
    if con.cursor().execute('''SELECT COUNT(*) FROM BBHs WHERE subject_genome =  ?  ''', (row[1],)).fetchall()[0][0]!=0:
        continue

    bbh_df = pd.read_csv("inputs/BBHs/"+row[2], sep="\t")
    con.cursor().executemany("""INSERT INTO BBHs(
                             subject_id, query_id, bit_score,
                             subject_genome, query_genome) 
                             VALUES(?, ?, ?, ?, ?)""", bbh_df.values.tolist())

# Create keys for subject and query gene IDs
con.cursor().execute("CREATE INDEX IF NOT EXISTS BBHs_subject_index ON BBHs(subject_genome)")
con.cursor().execute("CREATE INDEX IF NOT EXISTS BBHs_query_index ON BBHs(query_genome)")
con.commit()

<a class="anchor" id="orthogroups"></a>
# Insert OrthoFinder orthogroups
* [Go back to section 2](#section2)

In [6]:
con.cursor().execute("""DROP TABLE IF EXISTS orthogroups""")
con.commit()

con.cursor().execute("""CREATE TABLE IF NOT EXISTS orthogroups(
                     orthogroup text,
                     genome_id text,
                     gene_id text)""")

df = pd.read_csv(os.path.join("inputs","orthogroups", "Orthogroups.tsv"), sep="\t", index_col=0, low_memory=False)
df = df.applymap(lambda x: x.split(", ") if isinstance(x, str) else x)
orth_list = []
for col in df.columns:
    for ix in df.index:
        ls = df[col][ix]
        if type(ls) != list:
            continue
        for l in ls:
            orth_list.append([ix, col, l])

con.cursor().executemany("""INSERT INTO orthogroups(
                         orthogroup, genome_id, gene_id) 
                         VALUES(?, ?, ?)""", orth_list)

# Create keys for subject and query gene IDs
con.cursor().execute("CREATE INDEX IF NOT EXISTS orthogroup_gene_ids ON orthogroups(gene_id)")
con.cursor().execute("CREATE INDEX IF NOT EXISTS orthogroup_orthogroup_ids ON orthogroups(orthogroup)")
con.commit()

<a class="anchor" id="promoters"></a>
# Insert promoter sequences

* [Go back to section 2](#section2)

In [ ]:
con.cursor().execute("""CREATE TABLE IF NOT EXISTS promoter_seqs(
                     protein_id text,
                     genome_id text,
                     promoter_kind text,
                     promoter_length text,
                     promoter_sequence blob)""")

df = pd.read_csv("init/promoters.tsv", sep="\t")
for row in df.values:
    
    if con.cursor().execute('''SELECT COUNT(*) FROM promoter_seqs WHERE genome_id =  ?  ''', (row[0],)).fetchall()[0][0]!=0:
        continue

    fasta_file = gzip.open("inputs/promoters/"+row[5]+"_1kb_ATG.fasta.gz", mode='rt')
    promoter_seq_list = []
    for seq in SeqIO.parse(fasta_file, "fasta"):
        # TODO change db to number key to avoid duplicated name problems
        if row[0] == "ZmB73v3": 
            seq.id = seq.id + "v3"
        # Protein sequences are saved as as a Binary data type for compression
        promoter_seq_list.append([seq.id, row[0], "ATG", row[4], 
                                 sqlite3.Binary(zlib.compress(str(seq.seq).encode('utf-8')))])
    con.cursor().executemany("""INSERT INTO promoter_seqs(
                             protein_id, genome_id, promoter_kind,
                             promoter_length, promoter_sequence) 
                             VALUES(?, ?, ?, ?, ?)""", promoter_seq_list)
    
    fasta_file = gzip.open("inputs/promoters/"+row[5]+"_1kb_TSS.fasta.gz", mode='rt')
    promoter_seq_list = []
    for seq in SeqIO.parse(fasta_file, "fasta"):
        # TODO change db to number key to avoid duplicated name problems
        if row[0] == "ZmB73v3": 
            seq.id = seq.id + "v3"
        # Protein sequences are saved as as a Binary data type for compression
        promoter_seq_list.append([seq.id, row[0], "TSS", row[4], 
                                 sqlite3.Binary(zlib.compress(str(seq.seq).encode('utf-8')))])
    con.cursor().executemany("""INSERT INTO promoter_seqs(
                     protein_id, genome_id, promoter_kind,
                     promoter_length, promoter_sequence) 
                     VALUES(?, ?, ?, ?, ?)""", promoter_seq_list)

con.cursor().execute("CREATE INDEX IF NOT EXISTS promoter_index ON promoter_seqs(protein_id, promoter_kind)")
con.commit()

<a class="anchor" id="annotations"></a>
# Gene annotation insert
* [Go back to section 2](#section2)

In [ ]:
con.cursor().execute("""CREATE TABLE IF NOT EXISTS gene_annotations(
                     gene_id text,
                     genome_id text,
                     annotation_source text,
                     gene_annotation text)""")

df = pd.read_csv("init/annotation_list.tsv", sep="\t")

for row in df.values:
    
    if con.cursor().execute('''SELECT COUNT(*) FROM gene_annotations WHERE genome_id =  ?  ''', (row[0],)).fetchall()[0][0]!=0:
        continue

    
    annot = pd.read_csv("inputs/annotations/"+row[5], sep="\t")
    annot = annot.drop_duplicates(subset="locusName")

    if row[1]=="Arabidopsis thaliana":
        annot = annot[["locusName", "rice-defline"]]
    elif row[1] in ["Panicum hallii", "Pharus latifolius", "Solanum lycopersicum", "Vigna unguiculata"]:
        annot = annot[["locusName", "Best-hit-arabi-defline"]]        
    else:
        annot = annot[["locusName", "arabi-defline"]]
    annot.insert(1, 'genome_id', row[0])
    annot.insert(2, 'source', row[4])
    con.cursor().executemany("""INSERT INTO gene_annotations(
                         gene_id, genome_id, annotation_source, gene_annotation) 
                         VALUES(?, ?, ?, ?)""", annot.values.tolist())

con.cursor().execute("CREATE INDEX IF NOT EXISTS gene_annotations_index_gene_ids ON gene_annotations(gene_id)")
con.commit()

<a class="anchor" id="symbols"></a>
# Insert gene symbols
* [Go back to section 2](#section2)

In [ ]:
con.cursor().execute("""CREATE TABLE IF NOT EXISTS gene_symbols(
                     gene_id text,
                     genome_id text,
                     gene_symbol text)""")

df = pd.read_csv("init/symbols_list.tsv", sep="\t")

for row in df.values:
    if con.cursor().execute('''SELECT COUNT(*) FROM gene_symbols WHERE genome_id =  ?  ''', (row[0],)).fetchall()[0][0]!=0:
        continue

    annot = pd.read_csv("inputs/symbols/"+row[2], sep="\t")
    annot.insert(1, 'gene_id', row[0])
    con.cursor().executemany("""INSERT INTO gene_symbols(
                             gene_id, genome_id, gene_symbol) 
                             VALUES(?, ?, ?)""", annot.values.tolist())
con.cursor().execute("CREATE INDEX IF NOT EXISTS gene_symbols_index_gene_ids ON gene_symbols(gene_id)")
con.commit()

<a class="anchor" id="RNAseq"></a>
# Insert RNA-seq Files
* [Go back to section 2](#section2)

(Note: This part will need more work and is not currently in use)

In [ ]:
con.cursor().execute("""CREATE TABLE IF NOT EXISTS packages(
                     rowid integer,
                     name text,
                     version text,
                     settings text)""")

con.cursor().execute("""CREATE TABLE IF NOT EXISTS studies(
                     study_accession text, 
                     tax_id integer,
                     scientific_name text,
                     instrument_model text,
                     library_strategy text,
                     description text)""")

con.cursor().execute("""CREATE TABLE IF NOT EXISTS fastq(
                     run_accession text,
                     study_accession text,
                     read_count integer,
                     sample_alias text,
                     fastq_ftp text,
                     fastq_md5 text,
                     compression integer)""")

con.cursor().execute("""CREATE TABLE IF NOT EXISTS bam(
                     run_accession text,
                     study_accession text,
                     sample_alias text,
                     compression integer,
                     filter integer,
                     align integer,
                     sort integer)""")

def packages_insert(con, entities):
    cursorObj = con.cursor()
    cursorObj.execute("""INSERT INTO packages(
                         rowid, 
                         name,
                         version,
                         settings) 
                         VALUES(?, ?, ?, ?)""", entities)
    con.commit()
    
def studies_insert(con, entities):
    cursorObj = con.cursor()
    cursorObj.execute("""INSERT INTO studies(
                         study_accession, 
                         tax_id, 
                         scientific_name,
                         instrument_model,
                         library_strategy,
                         description) 
                         VALUES(?, ?, ?, ?, ?, ?)""", entities)
    con.commit()

# https://stackoverflow.com/questions/18219779/bulk-insert-huge-data-into-sqlite-using-python
def fastq_insert(con, entities):
    cursorObj = con.cursor()
    cursorObj.execute("""INSERT INTO fastq(
                         run_accession, 
                         study_accession, 
                         read_count, 
                         sample_alias,  
                         fastq_ftp,
                         fastq_md5,
                         compression) 
                         VALUES(?, ?, ?, ?, ?, ?, ?)""", entities)
    con.commit()
    
def bam_insert(con, entities):
    cursorObj = con.cursor()
    cursorObj.execute("""INSERT INTO bam(
                         run_accession,
                         study_accession,
                         sample_alias,
                         compression,
                         filter,
                         align,
                         sort) 
                         VALUES(?, ?, ?, ?, ?, ?, ?)""", entities)
    con.commit()

df = pd.read_csv("inputs/omics/packages.tsv", sep="\t")
for index, row in df.iterrows():
    packages_insert(con, entities=list(row))

df = pd.read_csv("inputs/omics/studies.tsv", sep="\t")
for index, row in df.iterrows():
    studies_insert(con, entities=list(row))

df = pd.read_csv("inputs/omics/fastq.tsv", sep="\t")
for index, row in df.iterrows():
    fastq_insert(con, entities=list(row))

df = pd.read_csv("inputs/omics/bam.tsv", sep="\t")
for index, row in df.iterrows():
    bam_insert(con, entities=list(row))

# SQNce Query Functions

##### GO enrichment query and testing functions

In [ ]:
from scipy.stats import  hypergeom
from statsmodels.stats.multitest import multipletests
rand_tps = ["Zm00001d021929","Zm00001d006678","Zm00001d008370","Zm00001d051416","Zm00001d017540","Zm00001d021410",
            "Zm00001d026188","Zm00001d034516","Zm00001d035106","Zm00001d036345","Zm00001d033547","Zm00001d035879",
            "Zm00001d036080","Zm00001d035881","Zm00001d045909","Zm00001d046750","Zm00001d012394","Zm00001d053084"]

In [ ]:
ls = []
for entity in rand_tps:
    cursorObj = con.cursor()
    cursorObj.execute('''SELECT gene_id, GO_id, GO_count 
                        FROM gene_GOs 
                        WHERE gene_id =  ?  ''', (entity,))
    # (name,) - need the comma to treat it as a single item and not list of letters
    selected = cursorObj.fetchall()
    if selected == []:
        continue
    else:
        ls.append(selected[0])
GO_df = pd.DataFrame(ls, columns=["gene_id", "GO_id", "GO_count"])

In [ ]:
cursorObj = con.cursor()
cursorObj.execute('''SELECT gene_number 
                    FROM genomes
                    WHERE genome_id =  ?  ''', ("ZmB73v4",))
# (name,) - need the comma to treat it as a single item and not list of letters
selected = cursorObj.fetchall()
genome_count = selected[0][0]

In [ ]:
def hyper_geom(row):
    hpd = hypergeom(row[2], row[1], row[3])
    p = hpd.pmf(row[0])
    return(p)

GO_df = pd.DataFrame(ls, columns=["gene_id", "GO_id", "GO_count"])
GO_counts = GO_df.groupby("GO_id").mean()["GO_count"]
GO_df = GO_df.groupby("GO_id").count()
GO_df["GO_count"] = GO_counts
GO_df["genome_count"] = genome_count
GO_df["group_count"] = len(rand_tps)
GO_df["pval"] = GO_df.apply(hyper_geom, axis=1)
GO_df["adj"] = list(multipletests(GO_df["pval"].values.tolist(), method="fdr_bh")[1])
GO_df["FC_enrichment"] = (GO_df["gene_id"] / GO_df["GO_count"]) / (GO_df["group_count"] / GO_df["genome_count"])

ls1 = []
for entity in GO_df.index:
    cursorObj = con.cursor()
    cursorObj.execute('''SELECT process, GO_short, GO_long 
                        FROM GO_basic
                        WHERE GO_id =  ?  ''', (entity,))
    # (name,) - need the comma to treat it as a single item and not list of letters
    selected = cursorObj.fetchall()
    if selected == []:
        continue
    else:
        ls1.append(selected[0])
GO_basic = pd.DataFrame(ls1, columns=["process", "GO_short", "GO_long"])
GO_df = GO_df.reset_index()
pd.concat([GO_df, GO_basic], axis=1)

##### Other functions

In [ ]:
rows = """Chr2\t12345\nChr3\t54354\nChr2\t5234354\n""".split("\n")
coordinate_list = [row.split("\t") for row in rows]
if coordinate_list[-1]==[""]:
    coordinate_list = coordinate_list[:-1]
for row in coordinate_list:
    if len(row) != 2:
        return(html.P("Number of columns is not 2. Use tab-seperated values."))


In [ ]:
# Query to find neighboring genes
def get_SNP_neighbors(genotype, chromsome, coordinate, distance):
    con = sqlite3.connect('SQNce.db')
    cursorObj = con.cursor()
    df = pd.read_sql_query('''SELECT * 
                     FROM gene_coordinates 
                     WHERE genotype_id = "{0}"
                     AND gene_chr = "{1}"
                     AND gene_start BETWEEN {2} AND {3}
                     
                     UNION ALL
                     
                     SELECT * 
                     FROM gene_coordinates 
                     WHERE genotype_id = "{0}"
                     AND gene_chr = "{1}"
                     AND gene_start BETWEEN {2} AND {3}
                     '''.format(genotype, chromsome, coordinate-distance, coordinate+distance), con)
    # Should check why it returns the same row twice, probably need to correct the query
    df = df.drop_duplicates()
    df.insert(0, 'Query', pd.Series(["_".join([chromsome, str(coordinate)]) for x in range(len(df.index))]))
    return(df)

df = pd.DataFrame()
for entity in [["Chr2", 19681637], ["Chr2", 1234564], ["Chr4", 1234564], ["Chr2", 19681638]]:
      df = pd.concat([df, get_SNP_neighbors("Arabidopsis", entity[0], entity[1], 10000)])  
#df.drop_duplicates(subset=["gene_id"]).reset_index()
#df1 = get_SNP_neighbors("Arabidopsis", "Chr2", 19681637, 10000)
#df1
#pd.concat([df1, df1])

In [ ]:
# Query to find neighboring genes and annotate them

df["annotation"] = annotation_select(con, df["gene_id"].to_list())
df

In [ ]:
def show_available_species(con):
    con = sqlite3.connect('SQNce.db')
    cursorObj = con.cursor()
    return(pd.read_sql_query("SELECT * FROM species", con))

show_available_species(con)

In [ ]:
cursorObj = con.cursor()
cursorObj.execute('''SELECT gene_id, gene_annotation 
                     FROM gene_annotations 
                     WHERE gene_id =  ?  ''', (entity,))
# (name,) - need the comma to treat it as a single item and not list of letters
selected = cursorObj.fetchall()[0]
od[selected[0]] = selected[1]

In [ ]:
# TODO add documentation to SQNce queries

def protein_seq_select(con, entity_list):
    od = OrderedDict()
    for entity in entity_list:
        cursorObj = con.cursor()
        cursorObj.execute('''SELECT protein_variant, gene_annotation 
                             FROM protein_seqs 
                             WHERE protein_variant =  ?  ''', (entity,))
        # (name,) - need the comma to treat it as a single item and not list of letters
        selected = cursorObj.fetchall()[0]
        record = SeqRecord(Seq(zlib.decompress(selected[1]).decode(encoding='UTF-8')), 
                           id=selected[0], name="", description="")
        od[selected[0]] = record
        with open("selected.fasta", 'w') as handle:
            SeqIO.write(od.values(), handle, 'fasta')

In [ ]:
def annotation_select(con, entity_list):
    ls = []
    for entity in entity_list:
        cursorObj = con.cursor()
        cursorObj.execute('''SELECT gene_id, gene_annotation 
                             FROM gene_annotations 
                             WHERE gene_id =  ?  ''', (entity,))
        # (name,) - need the comma to treat it as a single item and not list of letters
        selected = cursorObj.fetchall()
        if selected == []:
            ls.append("Gene not found")
        else:
            ls.append(selected[0][1])    
    return(ls)

con = sqlite3.connect('SQNce.db')
gene_list = ["Zm00001d010294", "dsa", "Sobic.002G128101", "AT2G34360", "Sobic.002G128101", "002G128101"]
df = pd.DataFrame({"name": gene_list, "annotation": annotation_select(con, gene_list) })
df.columns = ["GeneID", "annotation"]
df
#pd.DataFrame.from_dict(annotation_select(con, ["Zm00001d010294", "Sobic.002G128101", "AT2G34360"], "dict"), 
#                       orient="index", columns=["annotation"])


In [ ]:
annotation_select(con, ["Zm00001d010294", "Sobic.002G128101", "AT2G34360"])

In [ ]:
test = pd.DataFrame.from_dict(annotation_select(con, ["Zm00001d010294", "Sobic.002G128101", "AT2G34360"]), 
                              orient="index").reset_index()
test.columns = ["GeneID", "annotation"]
test 

# Example Queries

In [ ]:
con = sqlite3.connect('SQNce.db')
# Use the above query functions to parse SQNce with your gene lists 
input_value = [your_gene_list]
protein_seq_select(con, input_value)
con.close()

In [ ]:
con = sqlite3.connect('SQNce.db')
# Use the above query functions to parse SQNce with your gene lists 
input_value = [your_gene_list]
protein_seq_select(con, input_value)
con.close()

In [12]:
con = sqlite3.connect('SQNce.db')
cursorObj = con.cursor()
cursorObj.execute('''SELECT orthogroup, genome_id, gene_id
                    FROM orthogroups
                    WHERE orthogroup in (
                            SELECT orthogroup
                            FROM orthogroups
                            WHERE gene_id = ? ) ''', ("Zm00019ab05713",))
selected = cursorObj.fetchall()
selected

[]

In [18]:
con = sqlite3.connect('SQNce.db')
cursorObj = con.cursor()
pd.read_sql_query("""SELECT orthogroup, genome_id, gene_id
                    FROM orthogroups
                    WHERE orthogroup in (
                            SELECT orthogroup
                            FROM orthogroups
                            WHERE gene_id = '{0}' )""".format("Zm00036ab060450"), con)
#selected = cursorObj.fetchall()
#selected

,orthogroup,genome_id,gene_id
0,OG0018370,Bdistachyon,Bradi1g07500
1,OG0018370,Bhybridum,Brahy.D01G0094100
2,OG0018370,Bhybridum,Brahy.S02G0342900
3,OG0018370,Hmorex,HORVU.MOREX.r3.5HG0513060
4,OG0018370,Osativa,LOC_Os03g55770
5,OG0018370,Phallii,Pahal.9G074500
6,OG0018370,Pvirgatum,Pavir.9KG018258
7,OG0018370,Pvirgatum,Pavir.9NG041200
8,OG0018370,Sbicolor,Sobic.001G077700
9,OG0018370,Sitalica,Seita.9G078200


In [ ]:
[{ 'label': label, 'value': val} for label, val in [[1,2], [2,3]]]

In [ ]:
def distinct_db_vals(db, table, column, custom_vals=[], return_ls=False):
    # Input is the column to select and from which table
    # Returns a list of all values in a specific table from SQNce.db
    # Custom vals are added to the front using nested list of [label, value]
    ls = [{ 'label': label, 'value': val} for label, val in custom_vals]
    con = sqlite3.connect(db) # deploy with this
    cursorObj = con.cursor()
    distinct_df = pd.read_sql_query('''SELECT DISTINCT {0} 
                                       FROM {1}'''.format(column, table), con)
    if return_ls:
            return(distinct_df[column].to_list())
    for name in distinct_df[column]:
        ls.append({'label': name, 'value': name})
    return(ls)
distinct_db_vals("SQNce.db", "gene_coordinates", "genotype_id",[[1,2], [2,3]], True)

In [ ]:
def family_gene_select(gene_list):
    # Use an input list of genes to find their family assignments
    con = sqlite3.connect("SQNce.db")
    ls = []
    for gene in gene_list:
        cursorObj = con.cursor()
        cursorObj.execute('''SELECT protein_id, family_name 
                            FROM gene_families
                            WHERE protein_id =  ? ''', (gene,))
        selected = cursorObj.fetchall()
        if selected == []:
            ls.append("Gene not found")
        else:
            ls.append(selected[0][1])    
    return(ls)
family_gene_select(["Seita.5G010100", "test", "Zm00001d011673"])

In [ ]:
con = sqlite3.connect("SQNce.db") # deploy with this
cursorObj = con.cursor()
genotype = str("','".join(['Zea mays', "dsa",'Setaria italica']))
family = str("','".join(['Terpenoid synthases', 'Cytochrome P450']))
df = pd.read_sql_query("""SELECT protein_id, family_name 
                                   FROM gene_families
                                   WHERE species_id IN ('{0}') AND family_name IN ('{1}')""".format(genotype, family), con)
df

In [ ]:
selected = "subject_id"
entity_list = ["Zm00001d014121", "Zm00001d014134, ""Zm00001d014136"]
con = sqlite3.connect("SQNce.db") # deploy with this
cursorObj = con.cursor()
entity_list_str = str("','".join(entity_list))
df = pd.read_sql_query("""SELECT * 
                        FROM BBHs
                        WHERE {0} IN ('{1}')""".format(selected, entity_list_str), con)
df

In [ ]:
# This is a stupid function but it seems to work correctly.
df.sort_values(['bit_score'], ascending=False).groupby(["subject_id", "query_genotype"]).agg({"bit_score": "first", "query_id": "first",}).reset_index()